# Clean ways

In [582]:
import pandas as pd
import numpy as np

variable assignment

In [583]:
DICT_AREA_INDEX = {'Admin':3, 'Banking':3, 'CEO':3, 'Community':3}
DICT_TIME_INDEX = {'Admin':4, 'Banking':5, 'CEO':5}
DICT_CLASS_INDEX = {'Admin':5, 'Banking':6, 'CEO':4, 'Community':5}
DICT_SALARY_FLOOR = {'Admin':35000, 'Banking':53000, 'CEO':150000, 'Community':47000}
DICT_SALARY_CEILING = {'Admin':55000, 'Banking':85000, 'CEO':300000, 'Community':86000}
DICT_DUPLICATE_INDEX = {'Banking':4,'Community':4}
LIST_DUPLICATE = ['Banking','Community']
LIST_NO_CLASSIFICATION = ['CEO']
LIST_NO_POSTED_TIME = ['Community']
LIST_REDUNDANCY_INDEX = ['CEO','Community']


CLASSIFICATION_INDEX = 15
LOCATION_INDEX = 9
HOUR_TO_ANNUAL= 8*200

A function used in location_area() to clean repeated string input

In [584]:
# a tool function for cleaning repeated
def deduplication(x):
    mid = len(x)//2
    if x!='unknown':
        return x[:mid+1]
    else:
        return 'unknown'

In [585]:
def apply_salary(x):
    if 'classification:' in x:
        return 'unknown'
    else:
        return x

In [586]:
def apply_classification(x):
    if 'classification' in x:
        return x[CLASSIFICATION_INDEX:]
    else:
        return 'unknown'

In [587]:
def apply_posted_time(x):
    if 'd' in x:
        return -int(x[:-1])
    if 'm' in x:
        return -(int(x[:-1])*30)
    else:
        return 0

## clean_area

In [588]:
def clean_area_special(df, job_name): 
    if job_name in LIST_NO_CLASSIFICATION: 
        df[[3,'salary']]=df[3].str.split(',', n=1, expand=True)
    
    return df

In [589]:
def clean_area(df,area_index):
        
    df[['location','area']] = df[area_index].str.split('area:', n=1, expand=True)
    df['location'] = df['location'].apply(lambda x:x[LOCATION_INDEX:])
    df['area'].fillna(value='unknown',inplace = True)
    df['area'] = df['area'].apply(deduplication)
    df['location'].fillna(value='unknown',inplace = True)
    df['location'] = df['location'].apply(deduplication)
    
    return df


In [590]:
def clean_posted_time(df,time_index):
    df[['posted time','featurned at']] = df[time_index].str.split(',', expand = True)[[0,2]]
    df['featurned at'].fillna('unknown', inplace = True)
    df['posted time']=df['posted time'].str.extract('(\d+[a-z])')
    df['posted time'].fillna('unknown', inplace = True)
    df['posted time']=df['posted time'].astype('string')
    df['posted time']=df['posted time'].apply(apply_posted_time)
    
    return df

In [591]:
def clean_classification(df,class_index):
    df[['subclassification','classification']]=df[class_index].str.split('subClassification:', n=1, expand = True)
    df['classification'].fillna('unknown', inplace = True)
    df['classification'] = df['classification'].apply(deduplication)
    # salary derived from classification
    df['salary'] = df['subclassification'].apply(apply_salary)
    df['subclassification']=df['subclassification'].apply(apply_classification)
    df[['classification1','classification2','classification3']] = df['subclassification'].str.split('&', n=2, expand = True).fillna('unknown')
    
    return df

In [592]:
def clean_salary(df, job_name):
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace(' to ','-',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace('\d\%','',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace(',','',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace(' ','',regex=True)
    df['salary'].fillna('unknown' ,inplace = True)
    df['salary'].fillna('unknown', inplace=True)
    df[['low_salary','high_salary']] = df['salary'].str.split('-', n=1, expand = True).fillna('unknown')
    df['low_salary']=df['low_salary'].str.extract('(\d+k|\d+\.\d+|\d+ \d+|d+)')
    df['high_salary']=df['high_salary'].str.extract('(\d+k|\d+\.\d+|\d+ \d+|d+)')
    df['low_salary']=df[df['low_salary'].str.contains('\d', na=False)]['low_salary'].str.replace('k','000',regex=True)
    df['high_salary']=df[df['high_salary'].str.contains('\d', na=False)]['high_salary'].str.replace('k','000',regex=True)
    df['low_salary']=df['low_salary'].fillna('unknown')
    df['high_salary']=df['high_salary'].fillna('unknown')
    df.loc[df['low_salary']=='unknown', 'low_salary'] = df['high_salary']
    df.loc[df['high_salary']== 'unknown','high_salary']= df['low_salary']
    for r, row in enumerate(df['low_salary'].values):
        if row == 'unknown':
            df['low_salary'][r] = np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0]
    
    for r, row in enumerate(df['high_salary'].values):
        if row == 'unknown':
            df['high_salary'][r] = max(np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0],df['low_salary'][r])
        
    df['low_salary']=df['low_salary'].astype('float32')
    df['high_salary']= df['high_salary'].astype('float32')
    df.loc[df['low_salary']<50, 'low_salary'] = df['low_salary']*HOUR_TO_ANNUAL
    df.loc[df['high_salary']<50, 'high_salary'] = df['high_salary']*HOUR_TO_ANNUAL
    
    return df

In [593]:
def clean_redundancy(df, job_name):
    df.rename(columns ={0:'Job title',1:'website',2:'company name'}, inplace = True )
    if job_name not in LIST_NO_POSTED_TIME:
        df['company name'].fillna(df['featurned at'], inplace = True)
    #drop columns
    df.drop(DICT_AREA_INDEX[job_name], axis=1, inplace = True)
    if job_name not in LIST_NO_POSTED_TIME:
        df.drop(DICT_TIME_INDEX[job_name], axis=1, inplace = True)
    df.drop(DICT_CLASS_INDEX[job_name], axis=1, inplace = True)
    if job_name not in LIST_REDUNDANCY_INDEX:
        df.drop('subclassification', axis=1, inplace = True)
    if job_name not in LIST_NO_POSTED_TIME:
        df.drop('featurned at', axis=1, inplace = True)
    if job_name in LIST_DUPLICATE:
        df.drop(DICT_DUPLICATE_INDEX[job_name], axis=1, inplace=True)
        df['company name']=df[df['company name'].str.contains('at', na=False)]['company name'].str.replace('at ', '',  regex=True)
    if job_name in LIST_REDUNDANCY_INDEX:
        df.drop(0,axis=0,inplace=True) 
    df.drop('salary', axis=1, inplace = True)
       
    return df

In [594]:
##use for Admin or Jobs
def clean_jobs(file_path, job_name):
    df = pd.read_excel(file_path, header=None)
    
    df = clean_area_special(df, job_name)
    
    df = clean_area(df, DICT_AREA_INDEX[job_name])
    
    if job_name not in LIST_NO_POSTED_TIME:
        df = clean_posted_time(df, DICT_TIME_INDEX[job_name])
    
    if job_name not in LIST_NO_CLASSIFICATION:
        df = clean_classification(df, DICT_CLASS_INDEX[job_name])   
    
    df = clean_salary(df,job_name)
    
    df = clean_redundancy(df, job_name)
  
    return df

In [595]:
df_admin = clean_jobs("/Users/logan/Downloads/NZ_Admin_JOBS.xlsx",'Admin')

<ipython-input-592-bbef796a266a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['low_salary'][r] = np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0]
<ipython-input-592-bbef796a266a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['high_salary'][r] = max(np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0],df['low_salary'][r])


In [596]:
df_admin

,Job title,website,company name,location,area,posted time,classification,classification1,classification2,classification3,low_salary,high_salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of Plenty,Tauranga,0,Office Management,Administration,Office SupportAdministration,Office Support,43285.0,45215.0
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,Tauranga,0,Receptionists,Administration,Office SupportAdministration,Office Support,47962.0,54294.0
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,unknown,-4,Other,Administration,Office SupportAdministration,Office Support,36250.0,44347.0
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,Invercargill,0,Administrative Assistants,Administration,Office SupportAdministration,Office Support,38955.0,39467.0
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,Canterbury,Christchurch,-4,Client & Sales Administration,Administration,Office SupportAdministration,Office Support,39002.0,39002.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,Auckland,Auckland Central,-27,Client & Sales Administration,Administration,Office SupportAdministration,Office Support,49103.0,49103.0
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,Wellington,Wellington CentralWellington C,-27,unknown,unknown,unknown,unknown,39585.0,46959.0
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,Waikato,"HamiltonHamilton,Compe",-20,unknown,unknown,unknown,unknown,48591.0,48591.0
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,Waikato,"HamiltonHamilton,Compe",-28,unknown,unknown,unknown,unknown,48191.0,48191.0


In [597]:
df_banking = clean_jobs("/Users/logan/Downloads/NZ_Banking_JOBS.xlsx",'Banking')

<ipython-input-592-bbef796a266a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['low_salary'][r] = np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0]
<ipython-input-592-bbef796a266a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['high_salary'][r] = max(np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0],df['low_salary'][r])


In [598]:
df_banking

,Job title,website,company name,location,area,posted time,classification,classification1,classification2,classification3,low_salary,high_salary
0,Accounts Receivable,https://www.seek.co.nz/job/50568753?type=promo...,MTF Finance Mt Wellington,Auckland,Auckland Central,0,Credit,Banking,Financial ServicesBanking,Financial Services,78893.0,78893.0
1,Internal Audit Manager,https://www.seek.co.nz/job/50556333?type=promo...,Industrial and Commercial Bank of China (New Z...,Auckland,Auckland Central,0,Compliance & Risk,Banking,Financial ServicesBanking,Financial Services,56295.0,84922.0
2,Client Services Officer,https://www.seek.co.nz/job/50638706?type=stand...,NZ Funds Management Limited,Auckland,Auckland Central,0,Client Services,Banking,Financial ServicesBanking,Financial Services,63469.0,66680.0
3,Customer Banking Consultant - Invercargill,https://www.seek.co.nz/job/50637958?type=stand...,Westpac,Southland,"InvercargillInvercargill,Salary",-1,unknown,unknown,unknown,unknown,81552.0,81552.0
4,Private Wealth Assistant - Queenstown,https://www.seek.co.nz/job/50617226?type=stand...,Craigs Investment Partners,Otago,Queenstown & Wanaka,-5,Client Services,Banking,Financial ServicesBanking,Financial Services,82194.0,82194.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3879,Financial Advisor,https://www.seek.co.nz/job/50497173?type=stand...,NetYourJob,Auckland,"Rodney & North ShoreRodney & North Shore,",-26,unknown,unknown,unknown,unknown,56063.0,77969.0
3880,Senior Private Wealth Specialist,https://www.seek.co.nz/job/50526368?type=stand...,Debbie Graham & Associates Limited,Auckland,unknown,-20,Financial Planning,Banking,Financial ServicesBanking,Financial Services,59232.0,77822.0
3881,Financial Adviser (Auckland),https://www.seek.co.nz/job/50501103?type=stand...,Tyler Wren,Auckland,"Auckland CentralAuckland Central,Market",-25,unknown,unknown,unknown,unknown,67140.0,67370.0
3882,New Ventures Manager,https://www.seek.co.nz/job/50487878?type=stand...,Matthew Wood Search,Auckland,Auckland Central,-27,Corporate Finance & Investment Banking,Banking,Financial ServicesBanking,Financial Services,73317.0,73317.0


In [599]:
df_ceo = clean_jobs("/Users/logan/Downloads/NZ_CEO_JOBS.xlsx",'CEO')

<ipython-input-592-bbef796a266a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['low_salary'][r] = np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0]
<ipython-input-592-bbef796a266a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['high_salary'][r] = max(np.random.randint(DICT_SALARY_FLOOR[job_name],DICT_SALARY_CEILING[job_name],size=1)[0],df['low_salary'][r])


In [600]:
df_ceo

,Job title,website,company name,location,area,posted time,low_salary,high_salary
1,Director (x2) - Auckland Transport,https://www.seek.co.nz/job/50540328?type=promo...,Auckland Council,Auckland,Auckland Central,0,254583.0,256114.0
2,Director - Panuku Development,https://www.seek.co.nz/job/50540626?type=promo...,Auckland Council,Auckland,Auckland Central,0,179294.0,223723.0
3,Chief Executive,https://www.seek.co.nz/job/50623700?type=stand...,JacksonStone & Partners,Wellington,Porirua & Kapiti Coast,-4,189915.0,204177.0
4,Chief Executive,https://www.seek.co.nz/job/50623848?type=stand...,JacksonStone & Partners,Manawatu,Wanganui,-4,265743.0,278317.0
5,Chief Executive,https://www.seek.co.nz/job/50638864?type=stand...,Asset Recruitment Ltd,Waikato,Hamilton,0,282325.0,282325.0
...,...,...,...,...,...,...,...,...
2992,COMMERCIAL SERVICES MANAGEMENT OPPORTUNITY,https://www.seek.co.nz/job/50494754?type=stand...,Greg Tims & Associates,Waikato,Hamilton,-26,211609.0,211609.0
2993,Anaesthetic Technician Manager,https://www.seek.co.nz/job/50591866?type=stand...,Hawkes Bay District Health Board,Hawkes Bay,Hastings,-10,232290.0,232290.0
2994,Senior International Business Development Manager,https://www.seek.co.nz/job/50478483?type=stand...,The University of Auckland,Auckland,Auckland Central,-28,216885.0,295207.0
2995,General Manager | Tumu Whakahaere,https://www.seek.co.nz/job/50533152?type=stand...,Mana Recruitment,Bay of Plenty,Rest of Bay of Plenty,-19,168159.0,169935.0


In [601]:
df_Community = clean_jobs("/Users/logan/Downloads/NZ_CommunityServices_JOBS.xlsx",'Community')

In [602]:
df_Community

,Job title,website,company name,location,area,subclassification,classification,classification1,classification2,classification3,low_salary,high_salary
1,Kaiāwhina focused on Family Harm reduction,https://www.seek.co.nz/job/50617741?type=promo...,NaN,Gisborne,Gisborne,Community Services & DevelopmentCommunity Ser...,"Child Welfare, Youth & Family Services",Community Services,DevelopmentCommunity Services,Development,76809.0,76809.0
2,Community Arthritis Coordinator,https://www.seek.co.nz/job/50608532?type=promo...,NaN,Taranaki,New Plymouth,Community Services & DevelopmentCommunity Ser...,Community Development,Community Services,DevelopmentCommunity Services,Development,76843.0,76843.0
3,Family Worker,https://www.seek.co.nz/job/50639645?type=stand...,NaN,Auckland,Auckland Central,Community Services & DevelopmentCommunity Ser...,"Child Welfare, Youth & Family Services",Community Services,DevelopmentCommunity Services,Development,78051.0,78051.0
4,Community Facilitator,https://www.seek.co.nz/job/50614725?type=stand...,NaN,Waikato,Rest of Waikato,Community Services & DevelopmentCommunity Ser...,Aged & Disability Support,Community Services,DevelopmentCommunity Services,Development,52778.0,77573.0
5,Family Worker,https://www.seek.co.nz/job/50639529?type=stand...,NaN,Auckland,Waitakere & West Auckland,Community Services & DevelopmentCommunity Ser...,"Child Welfare, Youth & Family Services",Community Services,DevelopmentCommunity Services,Development,53566.0,53566.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60,Administrative Coordinator,https://www.seek.co.nz/job/50549089?type=stand...,NaN,Bay of Plenty,Tauranga,Community Services & DevelopmentCommunity Ser...,"Child Welfare, Youth & Family Services",Community Services,DevelopmentCommunity Services,Development,71784.0,71784.0
61,Lifestyle Coach - Kaitohutohu Ahua Noho,https://www.seek.co.nz/job/50633334?type=stand...,NaN,Auckland,Auckland Central,Community Services & DevelopmentCommunity Ser...,Aged & Disability Support,Community Services,DevelopmentCommunity Services,Development,71872.0,84096.0
62,Whanau Ora Navigator,https://www.seek.co.nz/job/50588310?type=stand...,NaN,Marlborough,Blenheim,Community Services & DevelopmentCommunity Ser...,Community Development,Community Services,DevelopmentCommunity Services,Development,65121.0,83390.0
63,Experienced Care and Support Worker,https://www.seek.co.nz/job/50615740?type=stand...,Creative Abilities and Associates Ltd,Auckland,"Rodney & North ShoreRodney & North Shore,$",unknown,unknown,unknown,unknown,unknown,47984.0,47984.0
